# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача: 

Обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок. Построить модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделайть выводы.

In [1]:
import time
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Подготовка

In [3]:
df = pd.read_csv('...')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
#уберем лишний столбец Unnamed 0
df = df.drop(columns=["Unnamed: 0"])

In [5]:
df.columns

Index(['text', 'toxic'], dtype='object')

In [6]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [8]:
#найдем уникальные значения для столбца "toxic", который будет у нас целевым признаком
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Выявлен сильный дисбаланс классов целевого признака, учтем это при обучении моделей 

In [9]:
#проверим на дубликаты
df.duplicated().sum()

0

In [10]:
#инициализация лемматайзера
lemmatizer = WordNetLemmatizer()

In [11]:
#функция для определения POS-тегов
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [12]:
#функция для очистки и лемматизации текста
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)  
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip() 
    text = text.lower()
    words = text.split()
    lemmatized_text = " ".join(lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words)
    return lemmatized_text

In [13]:
# применение функции к столбцу "text"
df['lemmatized_text'] = df['text'].apply(cleaning)

In [14]:
stopwords = set(stopwords.words('english'))

In [15]:
#просмотр результата
df.head()

,text,toxic,lemmatized_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,daww he match this background colour im seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not try to edit war it just ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestion on improv...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Промежуточный вывод: 
Данные были успешно прочитаны и подготовлены, удалён лишний столбец Unnamed: 0. Данные лемматизировали - привели слова к начальным формам, перезаписали столбец с текстом. 

## Обучение

In [16]:
#разделим данные на признаки и целевой столбец
X = df['lemmatized_text']
y = df['toxic'].values

#разделение на тренировочную и тестовую выборки в соотношении 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
count_tf_idf = TfidfVectorizer()

In [18]:
#векторизация текста
tfidf = TfidfVectorizer(stop_words=stopwords, max_features=10000)  

#преобразуем тренировочные и тестовые данные
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [19]:
#обучение модели LogisticRegression
model_lr = LogisticRegression(max_iter=1000,
                              random_state=42,
                              class_weight='balanced')

#подбор гиперпараметров
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

grid_search = GridSearchCV(model_lr, param_grid, cv=3, scoring='f1', verbose=1, n_jobs=-1)

start = time.time()
grid_search.fit(X_train_tfidf, y_train) 
end = time.time()


best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

#средний F1 по кросс-валидации
f1_lr = grid_search.best_score_  

print("\nЛучшие гиперпараметры:", best_params)
print("Среднее F1 Score на тренировочной выборке (кросс-валидация):", f1_lr)
print("Время обучения с подбором гиперпараметров:", end - start)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "



Лучшие гиперпараметры: {'C': 10, 'penalty': 'l2', 'solver': 'saga'}
Среднее F1 Score на тренировочной выборке (кросс-валидация): 0.7653394751438393
Время обучения с подбором гиперпараметров: 4089.626240968704


In [20]:
#обучение модели RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

#оценка модели с использованием кросс-валидации
start = time.time()
cv_scores_rf = cross_val_score(model_rf, X_train_tfidf, y_train, cv=5, scoring='f1', n_jobs=-1)
end = time.time()

f1_rf = cv_scores_rf.mean()

print("Random Forest:")
print("Время обучения:(c учетом кросс-валидации)", end - start)
print("F1 Score:", f1_rf)

Random Forest:
Время обучения:(c учетом кросс-валидации) 1141.9850716590881
F1 Score: 0.6776274110209461


In [21]:
#обучение модели SGDClassifier
model_sgd = SGDClassifier(random_state=42, class_weight='balanced')

#оценка модели с использованием кросс-валидации
start = time.time()
cv_scores_sgd = cross_val_score(model_sgd, X_train_tfidf, y_train, cv=5, scoring='f1')
end = time.time()

f1_sgd = cv_scores_sgd.mean()

# Вывод результатов
print("SGDClassifier:")
print("Среднее F1 Score (кросс-валидация):", f1_sgd)
print("Время обучения с кросс-валидацией:", end - start, "секунд")

SGDClassifier:
Среднее F1 Score (кросс-валидация): 0.7190087557090854
Время обучения с кросс-валидацией: 1.6156809329986572 секунд


In [22]:
results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'SGDClassifier'],
    'F1 Score': [f1_lr, f1_rf, f1_sgd]
})

results.sort_values(by='F1 Score', ascending=False)

,Model,F1 Score
0,Logistic Regression,0.765339
2,SGDClassifier,0.719009
1,Random Forest,0.677627


## Выводы

In [23]:
#финальная проверка одной модели на тестовой выборке
y_pred_test = best_model.predict(X_test_tfidf)
f1_test = f1_score(y_test, y_pred_test)

print("F1 Score на тестовой выборке:", f1_test)

F1 Score на тестовой выборке: 0.7748691099476439


Модель, удовлетворяющая требованию (F1 > 0.75), была найдена: значение F1 на тестовой выборке составило 0.775. В данных отсутствуют пропуски и дубликаты. Лучшая модель использует алгоритм Logistic Regression.